In [1]:
import utils.notebooks as nb
from utils.loader import DicomDataset3D
from torch.utils.data import DataLoader
from monai.losses import GeneralizedDiceLoss
import torch
from models.WNet2 import WNet
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

BATCH_SIZE = 4
EPOCHS = 3000
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = WNet(1).to(device)

train_dl = DataLoader(DicomDataset3D("data/train.csv"), batch_size=BATCH_SIZE)
validation_dl = DataLoader(DicomDataset3D("data/validation.csv"), batch_size=BATCH_SIZE)
test_dataloader = DataLoader(DicomDataset3D("data/test.csv"), batch_size=BATCH_SIZE)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
pos_weight = torch.tensor([300]).to(device)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device)
# criterion = nb.DiceBCELoss()

/home/casimir/UNI/WS_22/KAP/venv/lib/python3.11/site-packages/monai/networks/nets/unet.py:139: UserWarning: `len(strides) > len(channels) - 1`, the last 1 values of strides will not be used.
  warnings.warn(f"`len(strides) > len(channels) - 1`, the last {delta} values of strides will not be used.")


In [3]:
nb.train(model, train_dl, validation_dl, optimizer, criterion, EPOCHS, writer, model_name='wnet_bce')

epoch: 0
tensor(2.0784, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch: 1
tensor(1.6236, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch: 2
tensor(1.7460, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch: 3
tensor(1.4030, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch: 4
tensor(1.5808, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch: 5
tensor(1.3453, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch: 6
tensor(1.3244, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch: 7
tensor(1.2653, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch: 8
tensor(1.2721, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch: 9
tensor(1.2102, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch: 10
tensor(1.1914, devic

In [ ]:
torch.save(model.state_dict(), './state_dicts/easy_wnet.pk')
# model.load_state_dict(torch.load('./state_dicts/wnet_bce/best/wnet_bce_epoch_340_loss_0.0037.pk'))

<All keys matched successfully>

In [ ]:
nb.plot_predictions(test_dataloader, model)

In [ ]:

writer.add_graph(model, next(iter(train_dl))[0].to(device))
writer.flush()